In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


In [10]:
N=3
D=4
h=5
topic_s=1
topic_e=2
x = torch.randn(N, D)
W_list=torch.randn(N,h,topic_e)

In [27]:
def similarity(x1,x2):
    sim=torch.cosine_similarity(x1,x2,dim=0)
    x=sim*torch.cat((x1,x2),0)
    # print(torch.cat((x1,x2),0))
    return x

In [137]:
new_nodes=[]
for i in range(N):
    anchor_node=similarity(x[i],x[i])
    for j in range(N):
        if j==i:continue
        anchor_node_j=similarity(x[i],x[j])
        print(i,j)
        counts=0
        for t in range(topic_e-topic_s+1):
            print(W_list[i,:,t])
            print(W_list[j,:,t])
            max_si=torch.argmax(W_list[i,:,t])
            max_s2=torch.argmax(W_list[j,:,t])
            print ('max_nums',max_si,max_s2)
            if max_si==max_s2:
                counts+=1
        # print(counts)
        print(counts/(topic_e-topic_s))
        anchor_node = anchor_node + counts/(topic_e-topic_s)*anchor_node_j
    anchor_node = anchor_node/N
    # print(anchor_node)
    new_nodes.append(anchor_node.unsqueeze(0))
new_nodes=torch.cat([n for n in new_nodes],0)
print(new_nodes)

tensor([-0.7861,  0.3595, -1.5943,  0.1966, -0.7861,  0.3595, -1.5943,  0.1966])
tensor([-0.7861,  0.3595, -1.5943,  0.1966, -0.4323, -0.7272,  1.1543,  1.7036])
0 1
tensor([-0.2076,  0.4524,  0.7720,  0.8702, -0.8109])
tensor([ 1.8759, -0.0692, -0.0102, -0.4358,  1.4550])
max_nums tensor(3) tensor(0)
tensor([-0.3061,  0.7687,  1.3238, -0.4732, -1.6463])
tensor([-0.3900,  0.7136, -0.9679,  0.1849, -0.0050])
max_nums tensor(2) tensor(1)
0.0
tensor([-0.7861,  0.3595, -1.5943,  0.1966, -0.6052,  3.0320,  0.0388,  0.2366])
0 2
tensor([-0.2076,  0.4524,  0.7720,  0.8702, -0.8109])
tensor([ 0.8428,  0.2086, -0.7503, -0.6541,  0.7237])
max_nums tensor(3) tensor(0)
tensor([-0.3061,  0.7687,  1.3238, -0.4732, -1.6463])
tensor([ 1.6232,  0.1074,  0.6629, -0.8890,  1.2918])
max_nums tensor(2) tensor(0)
0.0
tensor([-0.4323, -0.7272,  1.1543,  1.7036, -0.4323, -0.7272,  1.1543,  1.7036])
tensor([-0.4323, -0.7272,  1.1543,  1.7036, -0.7861,  0.3595, -1.5943,  0.1966])
1 0
tensor([ 1.8759, -0.0692, -

In [131]:
print(x)
t1=x.repeat(N,1)
t2=x.repeat(1,N).reshape(-1,D)
t=torch.cat((t1,t2),1)
sim=torch.cosine_similarity(t1,t2)
sim=sim.unsqueeze(0)
print(sim)
print(t)
t=sim.T*t
print(t)
t=t.reshape(N,N,-1)
print(t)

tensor([[-0.7861,  0.3595, -1.5943,  0.1966],
        [-0.4323, -0.7272,  1.1543,  1.7036],
        [-0.6052,  3.0320,  0.0388,  0.2366]])
tensor([[ 1.0000, -0.3516,  0.2741, -0.3516,  1.0000, -0.2167,  0.2741, -0.2167,
          1.0000]])
tensor([[-0.7861,  0.3595, -1.5943,  0.1966, -0.7861,  0.3595, -1.5943,  0.1966],
        [-0.4323, -0.7272,  1.1543,  1.7036, -0.7861,  0.3595, -1.5943,  0.1966],
        [-0.6052,  3.0320,  0.0388,  0.2366, -0.7861,  0.3595, -1.5943,  0.1966],
        [-0.7861,  0.3595, -1.5943,  0.1966, -0.4323, -0.7272,  1.1543,  1.7036],
        [-0.4323, -0.7272,  1.1543,  1.7036, -0.4323, -0.7272,  1.1543,  1.7036],
        [-0.6052,  3.0320,  0.0388,  0.2366, -0.4323, -0.7272,  1.1543,  1.7036],
        [-0.7861,  0.3595, -1.5943,  0.1966, -0.6052,  3.0320,  0.0388,  0.2366],
        [-0.4323, -0.7272,  1.1543,  1.7036, -0.6052,  3.0320,  0.0388,  0.2366],
        [-0.6052,  3.0320,  0.0388,  0.2366, -0.6052,  3.0320,  0.0388,  0.2366]])
tensor([[-0.7861,  0.

In [140]:
# print(W_list,W_list.shape)
W_max=torch.argmax(W_list,dim=1)
# print(W_max,W_max.shape)
W1=W_max.repeat(N,1).reshape(-1,N,topic_e-topic_s+1)
W2=W_max.repeat(1,N).reshape(-1,N,topic_e-topic_s+1)
# print(W1)
# print(W2)
comp=torch.eq(W1,W2)
print(comp)
topic_count=torch.sum(comp,dim=2,keepdim=True)
print(topic_count)
topic_count=torch.div(topic_count.float(),topic_e-topic_s+1)
print(topic_count,topic_count.shape)
# comp=W1*comp
# comp=W2*comp
# print(comp)
# W_count=1-torch.nonzero(W_max>0)
# topic_count=torch.sum(W_count,dim=0)
# print(W_max,W_max.shape)
# print(W_count)
# print(topic_count)

tensor([[[ True,  True],
         [False, False],
         [False, False]],

        [[False, False],
         [ True,  True],
         [ True, False]],

        [[False, False],
         [ True, False],
         [ True,  True]]])
tensor([[[2],
         [0],
         [0]],

        [[0],
         [2],
         [1]],

        [[0],
         [1],
         [2]]])
tensor([[[1.0000],
         [0.0000],
         [0.0000]],

        [[0.0000],
         [1.0000],
         [0.5000]],

        [[0.0000],
         [0.5000],
         [1.0000]]]) torch.Size([3, 3, 1])


In [141]:
new_n=topic_count*t
print(new_n)
new_n=torch.mean(new_n,dim=0)
print(new_n)

tensor([[[1.0000],
         [0.0000],
         [0.0000]],

        [[0.0000],
         [1.0000],
         [0.5000]],

        [[0.0000],
         [0.5000],
         [1.0000]]])
tensor([[0.3333],
        [0.5000],
        [0.5000]])
